In [ ]:
!pip install ssspy

In [ ]:
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm.notebook import tqdm

In [ ]:
from ssspy.utils.dataset import download_sample_speech_data

In [ ]:
n_sources = 2
max_duration = 10
reverb_duration = 0.36
sisec2010_tag = "dev1_female3"
n_fft, hop_length = 4096, 2048

In [ ]:
waveform_src_img, sample_rate = download_sample_speech_data(
    n_sources=n_sources,
    sisec2010_tag=sisec2010_tag,
    max_duration=max_duration,
    reverb_duration=reverb_duration,
    conv=True,
)  # (n_channels, n_sources, n_samples)
waveform_mix = np.sum(waveform_src_img, axis=1)  # (n_channels, n_samples)

In [ ]:
for idx, waveform in enumerate(waveform_mix):
    print("Mixture: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
from ssspy.bss.mnmf import GaussMNMF as GaussMNMFBase

In [ ]:
class GaussMNMF(GaussMNMFBase):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.progress_bar = None

    def __call__(self, *args, n_iter: int = 100, **kwargs) -> np.ndarray:
        self.n_iter = n_iter

        return super().__call__(*args, n_iter=n_iter, **kwargs)

    def update_once(self) -> None:
        if self.progress_bar is None:
            self.progress_bar = tqdm(total=self.n_iter)

        super().update_once()

        self.progress_bar.update(1)

## w/ partitioning function

In [ ]:
mnmf = GaussMNMF(
    n_basis=30,
    n_sources=2,
    partitioning=True,
    rng=np.random.default_rng(42),
)
print(mnmf)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_est = mnmf(spectrogram_mix, n_iter=200)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
plt.figure()
plt.plot(mnmf.loss[10:])
plt.show()
plt.close()

## w/o partitioning function

In [ ]:
mnmf = GaussMNMF(
    n_basis=10,
    n_sources=2,
    partitioning=False,
    rng=np.random.default_rng(42),
)
print(mnmf)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_est = mnmf(spectrogram_mix, n_iter=500)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
plt.figure()
plt.plot(mnmf.loss[10:])
plt.show()
plt.close()